In [ ]:
!pip install -r "/content/drive/MyDrive/session_2/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 21.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 12.9 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.2/39.2 MB 37.3 MB/s eta 0:

In [ ]:
# Configure LlamaIndex Settings (Using OpenRouter - No OpenAI API Key needed)
def setup_llamaindex_settings():
    """
    Configure LlamaIndex with local embeddings and OpenRouter for LLM.
    This assignment focuses on vector database operations, so we'll use local embeddings only.
    """
    # Check for OpenRouter API key (for future use, not needed for this basic assignment)
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        print("ℹ️  OPENROUTER_API_KEY not found - that's OK for this assignment!")
        print("   This assignment only uses local embeddings for vector operations.")

    # Configure local embeddings (no API key required)
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5",
        trust_remote_code=True
    )

    print("✅ LlamaIndex configured with local embeddings")
    print("   Using BAAI/bge-small-en-v1.5 for document embeddings")

# Setup the configuration
setup_llamaindex_settings()

ℹ️  OPENROUTER_API_KEY not found - that's OK for this assignment!
   This assignment only uses local embeddings for vector operations.
✅ LlamaIndex configured with local embeddings
   Using BAAI/bge-small-en-v1.5 for document embeddings


In [ ]:
from llama_index.core import SimpleDirectoryReader

def load_documents_from_folder(folder_path: str):
    """
    Load documents from a folder using SimpleDirectoryReader.

    Args:
        folder_path (str): Path to the folder containing documents

    Returns:
        List of documents loaded from the folder
    """
    # Create SimpleDirectoryReader instance with recursive scan enabled
    reader = SimpleDirectoryReader(folder_path, recursive=True)

    # Load and return documents
    documents = reader.load_data()
    return documents


# Test the function after you complete it
test_folder = "/content/drive/MyDrive/session_2/data"
documents = load_documents_from_folder(test_folder)
print(f"Loaded {len(documents)} documents")


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 125MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Loaded 42 documents


In [ ]:
from pathlib import Path
from llama_index.vector_stores.lancedb import LanceDBVectorStore

def create_vector_store(db_path: str = "./vectordb", table_name: str = "documents"):
    """
    Create a LanceDB vector store for storing document embeddings.

    Args:
        db_path (str): Path where the vector database will be stored
        table_name (str): Name of the table in the vector database

    Returns:
        LanceDBVectorStore: Configured vector store
    """
    # Create the directory if it doesn't exist
    Path(db_path).mkdir(parents=True, exist_ok=True)

    # Create vector store
    vector_store = LanceDBVectorStore(uri=db_path, table_name=table_name)

    return vector_store


# Test the function after you complete it
vector_store = create_vector_store("./assignment_vectordb")
print(f"Vector store created: {vector_store is not None}")

Vector store created: True


In [ ]:
from typing import List
from llama_index.core import StorageContext, VectorStoreIndex

def create_vector_index(documents: List, vector_store):
    """
    Create a vector index from documents using the provided vector store.

    Args:
        documents: List of documents to index
        vector_store: LanceDB vector store to use for storage

    Returns:
        VectorStoreIndex: The created vector index
    """
    # Create storage context with the vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create index from documents
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context
    )

    return index


# Test the function after you complete it (will only work after previous functions are completed)
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print(f"Vector index created: {index is not None}")
else:
    print("Complete previous functions first to test this one")


Vector index created: True


In [ ]:
from typing import List
from llama_index.core import StorageContext, VectorStoreIndex

def create_vector_index(documents: List, vector_store):
    """
    Create a vector index from documents using the provided vector store.

    Args:
        documents: List of documents to index
        vector_store: LanceDB vector store to use for storage

    Returns:
        VectorStoreIndex: The created vector index
    """
    # Create storage context with the vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create index from documents
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context
    )

    return index


# Test the function after you complete it (will only work after previous functions are completed)
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print(f"Vector index created: {index is not None}")
else:
    print("Complete previous functions first to test this one")


Vector index created: True


In [ ]:
def search_documents(index, query: str, top_k: int = 3):
    """
    Search for relevant documents using the vector index.

    Args:
        index: Vector index to search
        query (str): Search query
        top_k (int): Number of top results to return

    Returns:
        List of retrieved document nodes
    """
    # Create retriever from index
    retriever = index.as_retriever(similarity_top_k=top_k)

    # Retrieve documents for the query
    results = retriever.retrieve(query)

    return results
# Test the function after you complete it (will only work after all previous functions are completed)
if 'index' in locals() and index is not None:
    test_query = "What are AI agents?"
    results = search_documents(index, test_query, top_k=2)
    print(f"Found {len(results)} results for query: '{test_query}'")
    for i, result in enumerate(results, 1):
        print(f"Result {i}: {result.text[:100] if hasattr(result, 'text') else 'No text'}...")
else:
    print("Complete all previous functions first to test this one")


Found 1 results for query: 'What are AI agents?'
Result 1: Architectures that leverage these techniques are more effective across a
variety of benchmarks and p...


In [ ]:
# Final test of the complete pipeline
print("🚀 Testing Complete Vector Database Pipeline")
print("=" * 50)

# Re-run the complete pipeline to ensure everything works
data_folder = "/content/drive/MyDrive/session_2/data"
vector_db_path = "./assignment_vectordb"

# Step 1: Load documents
print("\n📂 Step 1: Loading documents...")
documents = load_documents_from_folder(data_folder)
print(f"   Loaded {len(documents)} documents")

# Step 2: Create vector store
print("\n🗄️ Step 2: Creating vector store...")
vector_store = create_vector_store(vector_db_path)
print("   Vector store status:", "✅ Created" if vector_store else "❌ Failed")

# Step 3: Create vector index
print("\n🔗 Step 3: Creating vector index...")
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print("   Index status:", "✅ Created" if index else "❌ Failed")
else:
    index = None
    print("   ❌ Cannot create index - missing documents or vector store")

# Step 4: Test multiple search queries
print("\n🔍 Step 4: Testing search functionality...")
if index:
    search_queries = [
        "What are AI agents?",
        "How to evaluate agent performance?",
        "Italian recipes and cooking",
        "Financial analysis and investment"
    ]

    for query in search_queries:
        print(f"\n   🔎 Query: '{query}'")
        results = search_documents(index, query, top_k=2)

        if results:
            for i, result in enumerate(results, 1):
                text_preview = result.text[:100] if hasattr(result, 'text') else "No text available"
                score = f" (Score: {result.score:.4f})" if hasattr(result, 'score') else ""
                print(f"      {i}. {text_preview}...{score}")
        else:
            print("      No results found")
else:
    print("   ❌ Cannot test search - index not created")

print("\n" + "=" * 50)
print("🎯 Assignment Status:")
print(f"   Documents loaded: {'✅' if documents else '❌'}")
print(f"   Vector store created: {'✅' if vector_store else '❌'}")
print(f"   Index created: {'✅' if index else '❌'}")
print(f"   Search working: {'✅' if index else '❌'}")

if documents and vector_store and index:
    print("\n🎉 Congratulations! You've successfully completed the assignment!")
    print("   You've built a complete vector database with search functionality!")
else:
    print("\n📝 Please complete the TODO functions above to finish the assignment.")

🚀 Testing Complete Vector Database Pipeline

📂 Step 1: Loading documents...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


   Loaded 42 documents

🗄️ Step 2: Creating vector store...
   Vector store status: ✅ Created

🔗 Step 3: Creating vector index...
   Index status: ✅ Created

🔍 Step 4: Testing search functionality...

   🔎 Query: 'What are AI agents?'
      1. Architectures that leverage these techniques are more effective across a
variety of benchmarks and p... (Score: 0.6242)

   🔎 Query: 'How to evaluate agent performance?'
      1. steps, but the answers are limited to Yes/No responses [7]. As the industry continues to pivot towar... (Score: 0.6849)

   🔎 Query: 'Italian recipes and cooking'
      1. Spaghetti Carbonara, Italian, 20, Easy, Pasta, 450
Margherita Pizza, Italian, 45, Medium, Tomato, 32... (Score: 0.6380)

   🔎 Query: 'Financial analysis and investment'
      1. **Bond Ladders**: Interest rate risk management

### Aggressive Investors 🚀
1. **Growth Stocks**: Te... (Score: 0.5976)

🎯 Assignment Status:
   Documents loaded: ✅
   Vector store created: ✅
   Index created: ✅
   Search worki